### Sources

Blocking
- https://gist.github.com/metallurgix/8ee5262ed818730b5cc0



George Hotz
- (add yt video link)


Miduem Article
- https://vaibhaw-vipul.medium.com/matrix-multiplication-optimizing-the-code-from-6-hours-to-1-sec-70889d33dcfa


Neon Arm Instructions
- https://developer.arm.com/architectures/instruction-sets/intrinsics/#f:@navigationhierarchiessimdisa=[Neon]&f:@navigationhierarchiesreturnbasetype=[float]&f:@navigationhierarchieselementbitsize=[32]&q=vld1


Apple Accelerate
- https://developer.apple.com/documentation/accelerate/vdsp-library


AMX
- https://zhen8838.github.io/2024/04/23/mac-amx_en/


Loop unrolling
- https://en.wikipedia.org/wiki/Loop_unrolling#:~:text=Loop%20unrolling%2C%20also%20known%20as,or%20by%20an%20optimizing%20compiler.




| Description                              | Matrix Size | FLOP/s
|------------------------------------------|-------------|----------------
| Python without numpy                     | 256         | 0.0092 GFLOPS 
| st_np_matmul.py                          | 512         | 15.374713 GFLOPS
| st_PTB64.cpp                             | 1024        | 17.178770 GFLOPS
| np_matmul.py                             | 1024         | 52.210830 GFLOPS
| matrix_mult.cpp                          | 1024        | 34.398815 GFLOPS
| st_PTB1D.cpp                             | 1024        | 31.425362 GFLOPS
| st_PTB16.cpp                               | 1024        | 40.412571 GFLOPS
| PTB1D.cpp                                | 1024        | 127.062523 GFLOPS
| PTB4.cpp                                 | 1024        | 101.024773 GFLOPS
| PTB8.cpp                                 | 1024        | 63.231953 GFLOPS
| PTB16.cpp                                | 1024        | 194.324829 GFLOPS
| PTB32.cpp                                | 1024        | 94.407333 GFLOPS
| accelerate.cpp                           | 8192        | 1326.189209 GFLOPS


(blocksize over 16 make yields less FLOPs)
(-O3 is faster than -O2 for me)


Run the best code
`clang++ PTB.cpp -O3 -march=native -ffast-math -Xpreprocessor -fopenmp -I/opt/homebrew/include -L/opt/homebrew/lib -lomp -o compiled/PTB && ./compiled/PTB`


Accelerate

`clang++ -O3 -DACCELRATE_NEW_LAPACK -DACCELERATE_LAPACK_ILP64 accelerate.cpp -framework Accelerate -o ./compiled/accelerate && ./compiled/accelerate`


### Questions

Blocking helps with cache, but why?

#pragma omp parallel for private(bj, bk, i, j, k) shared(A, B_trans, C) - what does this do *exacly*? why is bi not included?